In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np


In [9]:
hex_proof = gpd.read_file('./synced-data/rwa_travel_time_hex-8.geojson')
hex = hex_proof.copy()

In [17]:
school_bins = [30, 45, 60]
health_bins = [45, 90, 135]
hospital_bins = [60, 120, 180]
market_bins = [60, 120, 180]

In [53]:
scoring_rubric =[[0, 0, 0, 0], [2, 0, 0, 0], [4, 2, 0, 0], [8, 4, 2, 0]]

def calculate_impact_score(before_travel_time, after_travel_time, bins):
    # Calculate the bin indices for before and after travel times
    before_bin = np.digitize(before_travel_time, bins)
    after_bin = np.digitize(after_travel_time, bins)
    
    # Initialize an array to store impact scores
    impact_scores = np.zeros(len(before_travel_time))
    
    # Loop through each row and calculate the impact score
    for i in range(len(before_travel_time)):
        impact_scores[i] = scoring_rubric[before_bin[i]][after_bin[i]]
    
    return impact_scores

In [ ]:
hex['all_education_potential_impact'] = calculate_impact_score(hex['travel_time_no_sites_all_education_facilities_fixed'], hex['travel_time_all_education_facilities_fixed'], school_bins)
hex['all_education_current_impact'] = calculate_impact_score(hex['travel_time_no_sites_all_education_facilities_fixed'], hex['travel_time_constructed_sites_all_education_facilities_fixed'], school_bins)
hex['all_health_potential_impact'] = calculate_impact_score(hex['travel_time_no_sites_all_health_facilities_optimal'], hex['travel_time_all_health_facilities_optimal'], health_bins)
hex['all_health_current_impact'] = calculate_impact_score(hex['travel_time_no_sites_all_health_facilities_optimal'], hex['travel_time_constructed_sites_all_health_facilities_optimal'], health_bins)
hex['health_centers_potential_impact'] = calculate_impact_score(hex['travel_time_no_sites_health_centers_optimal'], hex['travel_time_health_centers_optimal'], health_bins)
hex['health_centers_current_impact'] = calculate_impact_score(hex['travel_time_no_sites_health_centers_optimal'], hex['travel_time_constructed_sites_health_centers_optimal'], health_bins)
hex['health_posts_potential_impact'] = calculate_impact_score(hex['travel_time_no_sites_health_posts_optimal'], hex['travel_time_health_posts_optimal'], health_bins)
hex['health_posts_current_impact'] = calculate_impact_score(hex['travel_time_no_sites_health_posts_optimal'], hex['travel_time_constructed_sites_health_posts_optimal'], health_bins)
hex['major_hospitals_potential_impact'] = calculate_impact_score(hex['travel_time_no_sites_major_hospitals_optimal'], hex['travel_time_major_hospitals_optimal'], hospital_bins)
hex['major_hospitals_current_impact'] = calculate_impact_score(hex['travel_time_no_sites_major_hospitals_optimal'], hex['travel_time_constructed_sites_major_hospitals_optimal'], hospital_bins)
hex['primary_schools_potential_impact'] = calculate_impact_score(hex['travel_time_no_sites_primary_schools_fixed'], hex['travel_time_primary_schools_fixed'], school_bins)
hex['primary_schools_current_impact'] = calculate_impact_score(hex['travel_time_no_sites_primary_schools_fixed'], hex['travel_time_constructed_sites_primary_schools_fixed'], school_bins)
hex['secondary_schools_potential_impact'] = calculate_impact_score(hex['travel_time_no_sites_secondary_schools_fixed'], hex['travel_time_secondary_schools_fixed'], school_bins)
hex['secondary_schools_current_impact'] = calculate_impact_score(hex['travel_time_no_sites_secondary_schools_fixed'], hex['travel_time_constructed_sites_secondary_schools_fixed'], school_bins)
hex['semi_dense_urban_potential_impact'] = calculate_impact_score(hex['travel_time_no_sites_semi_dense_urban_optimal'], hex['travel_time_semi_dense_urban_optimal'], market_bins)
hex['semi_dense_urban_current_impact'] = calculate_impact_score(hex['travel_time_no_sites_semi_dense_urban_optimal'], hex['travel_time_constructed_sites_semi_dense_urban_optimal'], market_bins)

In [57]:
hex['total_potential_impact'] = hex['health_centers_potential_impact'] + hex['health_posts_potential_impact'] + hex['major_hospitals_potential_impact'] + hex['primary_schools_potential_impact'] + hex['secondary_schools_potential_impact'] + hex['semi_dense_urban_potential_impact']
hex['total_current_impact'] = hex['health_centers_current_impact'] + hex['health_posts_current_impact'] + hex['major_hospitals_current_impact'] + hex['primary_schools_current_impact'] + hex['secondary_schools_current_impact'] + hex['semi_dense_urban_current_impact']
hex['total_school_current_impact'] =  hex['primary_schools_current_impact'] + hex['secondary_schools_current_impact']
hex['total_school_potential_impact'] =  hex['primary_schools_potential_impact'] + hex['secondary_schools_potential_impact']
hex['total_health_current_impact'] = hex['health_centers_current_impact'] + hex['health_posts_current_impact'] + hex['major_hospitals_current_impact']
hex['total_health_potential_impact'] = hex['health_centers_potential_impact'] + hex['health_posts_potential_impact'] + hex['major_hospitals_potential_impact']

In [63]:
hex.to_file('./synced-data/rwa_travel_time_hex-8.geojson', driver='GeoJSON')

# trimming down data file size

In [8]:
large_file = gpd.read_file('https://fikamap-web-app.s3.us-west-1.amazonaws.com/data/rwa_travel_time_hex-8.geojson')

In [4]:
pd.set_option('display.max_columns', None)

In [6]:

large_file = large_file.drop("row_col", axis=1)

In [7]:
# write large_file to geojson
large_file.to_file('./unsynced-data/rwanda/rwa_travel_time_hex-8-small.geojson', driver='GeoJSON')

In [11]:
large_file

,h3-index,row_col,female_educational_attainment_mean,females_0_4,females_0_9,females_10_14,females_15_49,females_50_64,females_5_9,females_65_plus,male_educational_attainment_mean,males_0_4,males_0_9,males_10_14,males_15_49,males_50_64,males_5_9,males_65_plus,pop_0_4,pop_0_9,pop_10_14,pop_15_49,pop_50_64,pop_5_9,pop_65_plus,population,pregnancies,births,rwi,underweight,time_delta_constructed_sites_all_education_facilities_fixed,time_delta_constructed_sites_all_health_facilities_optimal,time_delta_constructed_sites_health_centers_optimal,time_delta_constructed_sites_health_posts_optimal,time_delta_constructed_sites_major_hospitals_optimal,time_delta_constructed_sites_primary_schools_fixed,time_delta_constructed_sites_secondary_schools_fixed,time_delta_constructed_sites_semi_dense_urban_optimal,time_delta_no_sites_all_education_facilities_fixed,time_delta_no_sites_all_health_facilities_optimal,time_delta_no_sites_health_centers_optimal,time_delta_no_sites_health_posts_optimal,time_delta_no_sites_major_hospitals_optimal,time_delta_no_sites_primary_schools_fixed,time_delta_no_sites_secondary_schools_fixed,time_delta_no_sites_semi_dense_urban_optimal,travel_time_all_education_facilities_fixed,travel_time_all_health_facilities_optimal,travel_time_constructed_sites_all_education_facilities_fixed,travel_time_constructed_sites_all_health_facilities_optimal,travel_time_constructed_sites_health_centers_optimal,travel_time_constructed_sites_health_posts_optimal,travel_time_constructed_sites_major_hospitals_optimal,travel_time_constructed_sites_primary_schools_fixed,travel_time_constructed_sites_secondary_schools_fixed,travel_time_constructed_sites_semi_dense_urban_optimal,travel_time_health_centers_optimal,travel_time_health_posts_optimal,travel_time_major_hospitals_optimal,travel_time_no_sites_all_education_facilities_fixed,travel_time_no_sites_all_health_facilities_optimal,travel_time_no_sites_health_centers_optimal,travel_time_no_sites_health_posts_optimal,travel_time_no_sites_major_hospitals_optimal,travel_time_no_sites_primary_schools_fixed,travel_time_no_sites_secondary_schools_fixed,travel_time_no_sites_semi_dense_urban_optimal,travel_time_primary_schools_fixed,travel_time_secondary_schools_fixed,travel_time_semi_dense_urban_optimal,all_education_potential_impact,all_education_current_impact,all_health_potential_impact,all_health_current_impact,health_centers_potential_impact,health_centers_current_impact,health_posts_potential_impact,health_posts_current_impact,major_hospitals_potential_impact,major_hospitals_current_impact,primary_schools_potential_impact,primary_schools_current_impact,secondary_schools_potential_impact,secondary_schools_current_impact,semi_dense_urban_potential_impact,semi_dense_urban_current_impact,total_potential_impact,total_current_impact,total_school_current_impact,total_school_potential_impact,total_health_current_impact,total_health_potential_impact,geometry
0,886ad80001fffff,1334_1576&1335_1573&1335_1574&1335_1575&1335_1...,4.6503,17,36,18,71,12,19,4,4.9094,16,36,19,74,12,19,3,33,72,37,146,24,38,8,290,20,14,0.0152,0.0814,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0,67,112,67,112,199,112,332,67.240000,162.600000,185,199,112,332,67,112,199,112,332,67.240000,162.600000,185,67.240000,162.600000,185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((30.17441 -2.16518, 30.17633 -2.16048..."
1,886ad80003fffff,1338_1586&1338_1587&1338_1588&1339_1583&1339_1...,4.5309,25,56,29,116,19,30,7,4.8748,26,57,30,111,16,31,5,52,114,59,228,35,62,12,451,34,24,-0.0153,0.0877,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0,60,104,60,104,180,104,341,60.731707,130.878049,181,180,104,341,60,104,180,104,341,60.731707,130.878049,181,60.731707,130.878049,181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((30.18272 -2.16847, 30.18464 -2.16378..."
2,886ad80005fffff,1343_1564&1344_1563&1344_1564&1344_1572&1344_1...,4.4477,12,29,17,64,10,17,3,4.8480,13

In [13]:
# row_col = just the hex_id and h3-index columns from large_file
row_col = large_file[['h3-index', 'row_col']]

In [15]:
# write dataframe to json
row_col.to_json('./unsynced-data/rwanda/rwa_travel_time_hex-8-row_col.json', orient='records')